# Prediction

In [1]:
from config import config
config = config.Configuration()

### Model

In [2]:
from core.losses import tversky, accuracy, dice_coef, dice_loss, specificity, sensitivity
from core.optimizers import adaDelta
import keras
from keras.models import load_model

OPTIMIZER = adaDelta
LOSS = tversky
# check if model_path is already defined. This is the case when you run the pipeline and not this notebook alone
if not "model_path" in locals(): model_path = config.filepath_model
model = load_model(model_path, custom_objects={'tversky': LOSS, 
                                               'dice_coef': dice_coef, 
                                               'dice_loss': dice_loss, 
                                               'accuracy': accuracy , 
                                               'specificity': specificity, 
                                               'sensitivity': sensitivity}, 
                                               compile=False)

2022-04-14 00:30:20.922567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-04-14 00:30:20.922602: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-04-14 00:30:20.923270: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [3]:
def scale_up_color_val(array):
    return [(num*255).astype(np.uint8) for num in array]

def scale_down_color_val(array):
    return [(num/255) for num in array]

def visualize_res(x):
    plt.imshow(scale_up_color_val(X_pred[x]))
    plt.show()
    plt.imshow(scale_up_color_val(y_pred[x]))
    plt.show()

In [ ]:
%%time
import os.path
import numpy as np
filepath_pred_patches = f"{config.path_pred}patches_{config.pred_imgs[0][:-4]}.npy"
# check if satellite image exists and the patches have not been generated previously
if os.path.isfile(config.path_satellite + config.pred_imgs[0]) and not os.path.isfile(filepath_pred_patches):
    %run "patch_creation_pred.py"
    print("Patches Created")
elif os.path.isfile(filepath_pred_patches):
    X_pred = np.load(filepath_pred_patches)

y_pred = model.predict(scale_down_color_val(X_pred))
filepath_pred_y = f"{config.path_patches}y_pred_patches_{filename[:-4]}.npy"
np.save(filepath_pred_y, y_pred)

In [ ]:
visualize_res(10)